<h1><center> Group Assignment: Bird Classifier  </center></h1>

In [2]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available. Running on Apple GPU.")
else:
    print("MPS backend is not available. Running on CPU.")


MPS backend is available. Running on Apple GPU.


In [4]:
device = torch.device("mps")  # Use MPS for Apple Silicon
x = torch.tensor([1.0, 2.0, 3.0], device=device)
print("Tensor on MPS:", x)


Tensor on MPS: tensor([1., 2., 3.], device='mps:0')


In [60]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn, optim
from PIL import Image

# Step 1: Read and preprocess the CSV
data = pd.read_csv('data/train_images.csv')

# Prepend 'data' to paths if not already present
data['image_path'] = data['image_path'].apply(lambda x: f"data{x}" if not x.startswith("data") else x)

# Verify the first few paths
print(data['image_path'].head())

# Check if files exist
for path in data['image_path'].head():
    print(f"{path} exists: {os.path.exists(path)}")

# Step 2: Define a PyTorch Dataset    
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        label = torch.tensor(self.labels[idx]).long()  # Convert label to tensor
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.image_paths)

# Define transformations
from torchvision import transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])


# Create dataset and dataloader
image_paths = data['image_path'].values
labels = data['label'].values - 1  # Ensure labels are zero-indexed (0 to 199)
print(labels)

dataset = CustomDataset(image_paths, labels, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


0    data/train_images/1.jpg
1    data/train_images/2.jpg
2    data/train_images/3.jpg
3    data/train_images/4.jpg
4    data/train_images/5.jpg
Name: image_path, dtype: object
data/train_images/1.jpg exists: True
data/train_images/2.jpg exists: True
data/train_images/3.jpg exists: True
data/train_images/4.jpg exists: True
data/train_images/5.jpg exists: True
[  0   0   0 ... 199 199 199]


In [ ]:
import torch.nn as nn

# Step 3: Define the PyTorch model
import torch
import torch.nn as nn
import torch.optim as optim

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Define the layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Conv layer 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Conv layer 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer
        self.fc1 = nn.Linear(64 * 37 * 37, 512)  # Adjusted fully connected layer 1 size
        self.fc2 = nn.Linear(512, 200)  # Output layer (for 200 classes)

    def forward(self, x):
        # Define the forward pass
        x = self.pool(torch.relu(self.conv1(x)))  # Apply ReLU after conv1 and max pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Apply ReLU after conv2 and max pooling

        # Flatten the tensor for fully connected layer (adjust this part)
        x = x.view(x.size(0), -1)  # Flatten dynamically based on the current shape

        x = torch.relu(self.fc1(x))  # Apply ReLU after the first fully connected layer
        x = self.fc2(x)  # Output layer
        return x

In [62]:

labels = torch.tensor(labels).long()  # Ensure labels are in the correct type

# Step 4: Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)  # Move the model to the appropriate device (CPU or GPU)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [63]:
print(torch.unique(labels))


tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [64]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:

        images = images.to(device)
        labels = torch.tensor(labels).long().to(device)
        # images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get class with highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


/var/folders/w9/03c_s4jn509125_l99795s6c0000gn/T/ipykernel_1294/1925588254.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).long().to(device)


Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: tor

In [67]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 1: Define the transformations for the test data (same as for training)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Step 2: Load the test dataset using ImageFolder
test_dir = 'data/test_images'  # Path to your test image directory

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # No shuffle for evaluation

# Step 3: Evaluate the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get class with highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")


Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: torch.Size([32, 64, 37, 37])
Shape before flattening: tor